Optimizing Online Sports Retail Revenue Project:

In this notebook I will play the role of a product analyst for an online sports clothing company. The company is interested in how it can improve revenue. I will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.

First I'm going to check the total number of products and non-missing values in the data.

In [2]:
%%sql
postgresql:///sports

SELECT COUNT(*) AS total_rows, 
    COUNT(i.description) AS count_description, 
    COUNT(f.listing_price) AS count_listing_price, 
    COUNT(t.last_visited) AS count_last_visited 
FROM info AS i
INNER JOIN finance AS f
    ON i.product_id = f.product_id
INNER JOIN traffic AS t
    ON t.product_id = f.product_id;

1 rows affected.


total_rows,count_description,count_listing_price,count_last_visited
3179,3117,3120,2928


There is a total of 3179 products and last visited is the only column missing more than 5% of values.
Next I will be digging into pricing. I will see how the price points for Nike and Adidas differ to get an idea of the company's stock range and customer market for 2 of the biggest companies in the sports clothing industry.

In [4]:
%%sql

SELECT b.brand, f.listing_price::integer, COUNT(f.*)
FROM brands AS b
INNER JOIN finance AS f
ON b.product_id = f.product_id
WHERE f.listing_price > 0
GROUP BY b.brand, f.listing_price
ORDER BY f.listing_price DESC;

 * postgresql:///sports
77 rows affected.


brand,listing_price,count
Adidas,300,2
Adidas,280,4
Adidas,240,5
Adidas,230,8
Adidas,220,11
Adidas,200,8
Nike,200,1
Adidas,190,7
Nike,190,2
Adidas,180,34


There are 77 unique prices for Nike and Adidas products so to better analyze this data I will assign labels to different price ranges and check the total revenue for each price range.

In [6]:
%%sql

SELECT b.brand, COUNT(f.*), SUM(revenue) AS total_revenue,
    CASE WHEN f.listing_price < 42 THEN 'Budget'
         WHEN f.listing_price >= 42 AND f.listing_price < 74 THEN 'Average'
         WHEN f.listing_price >= 74 AND f.listing_price < 129 THEN 'Expensive'
         ELSE 'Elite' END AS price_category
FROM brands AS b
INNER JOIN finance AS f
ON b.product_id = f.product_id
WHERE b.brand IS NOT NULL
GROUP BY b.brand, price_category
ORDER BY total_revenue DESC;

 * postgresql:///sports
8 rows affected.


brand,count,total_revenue,price_category
Adidas,849,4626980.069999999,Expensive
Adidas,1060,3233661.060000001,Average
Adidas,307,3014316.8299999987,Elite
Adidas,359,651661.1200000002,Budget
Nike,357,595341.0199999992,Budget
Nike,82,128475.59000000003,Elite
Nike,90,71843.15000000004,Expensive
Nike,16,6623.5,Average


Adidas is generating more revenue than Nike in every category specifically in the "Expensive" category so the company could potentially increase revenue by caring more stock in that category.

Next I will account for discount to see if there is a difference in the amount of discount between brands.

In [8]:
%%sql

SELECT b.brand, AVG(f.discount) * 100 AS average_discount
FROM brands AS b
INNER JOIN finance AS f 
ON b.product_id = f.product_id
GROUP BY b.brand
HAVING b.brand IS NOT NULL
ORDER BY average_discount;


 * postgresql:///sports
2 rows affected.


brand,average_discount
Nike,0.0
Adidas,33.452427184465606


Looks like there is no discount offered on Nike products but Adidas have a average discount of around 33%. To improve revenue they could try reducing the amount of discount offered on Adidas products and offer a small discount to Nike products.

Next I will check the strength of correlation between revenue and reviews.

In [10]:
%%sql

SELECT CORR(r.reviews, f.revenue) AS review_revenue_corr
FROM reviews AS r
INNER JOIN finance AS f
ON r.product_id = f.product_id;

 * postgresql:///sports
1 rows affected.


review_revenue_corr
0.6518512283481301


There seemd to be a strong correlation between reviews and revenue, if there store can get more reviews they may able to increase sales on items will high review counts.

I'm going to look at the volume of reviews per month to see if there's any trends or gaps.

In [14]:
%%sql

SELECT b.brand, DATE_PART('month', t.last_visited) AS month, COUNT(r.*) AS num_reviews
FROM brands AS b
INNER JOIN traffic AS t 
ON b.product_id = t.product_id
INNER JOIN reviews AS r 
 ON t.product_id = r.product_id
GROUP BY b.brand, month
HAVING b.brand IS NOT NULL AND DATE_PART('month', t.last_visited) IS NOT NULL
ORDER BY b.brand, month;

 * postgresql:///sports
24 rows affected.


brand,month,num_reviews
Adidas,1.0,253
Adidas,2.0,272
Adidas,3.0,269
Adidas,4.0,180
Adidas,5.0,172
Adidas,6.0,159
Adidas,7.0,170
Adidas,8.0,189
Adidas,9.0,181
Adidas,10.0,192


The volume of product reviews are higher in the first quarter year but then take a dip. The store could make an emphasis to encourage customers to review their products throughout the entire year.

Now I will switch focus to analyzing all footwear being sold at the store. I will use a CTE to do this because there are no labels for product types provided.

In [16]:
%%sql

WITH footwear AS
    (SELECT i.description, f.revenue
    FROM info AS i
    INNER JOIN finance AS f 
    ON i.product_id = f.product_id
    WHERE i.description ILIKE '%shoe%'
        OR i.description ILIKE '%trainer%'
        OR i.description ILIKE '%foot%'
        AND i.description IS NOT NULL)
SELECT COUNT(*) AS num_footwear_products, percentile_disc(0.5) WITHIN GROUP (ORDER BY revenue) AS median_footwear_revenue
FROM footwear;

 * postgresql:///sports
1 rows affected.


num_footwear_products,median_footwear_revenue
2700,3118.36


I know there is a total of 3179 products in the database so footwear makes up around 85% of the company's stock and generate a median revenue of over $3000 dollars.

Lastly I will check the number of products and median revenue for non-footwear products.

In [18]:
%%sql

WITH footwear AS
    (SELECT i.description, f.revenue
    FROM info AS i
    INNER JOIN finance AS f 
    ON i.product_id = f.product_id
    WHERE i.description ILIKE '%shoe%'
        OR i.description ILIKE '%trainer%'
        OR i.description ILIKE '%foot%'
        AND i.description IS NOT NULL)
SELECT COUNT(i.*) AS num_clothing_products, percentile_disc(0.5) WITHIN GROUP (ORDER BY f.revenue) AS median_clothing_revenue
FROM info AS i
INNER JOIN finance AS f
ON i.product_id = f.product_id
WHERE description NOT IN (SELECT description FROM footwear);

 * postgresql:///sports
1 rows affected.


num_clothing_products,median_clothing_revenue
417,503.82


Conclusion:
-Adidas products is major source of revenue and is suprerior to Nike in every price category.
-Adidas "Expensive" category drives the most revenue so shifting stock towards these products can increase revenue.
-Decreasing the amount of discount on Adidas products and slighty increasing dicounts on Nike producs may increase revenue.
-Products with reviews generate more revenue so encouraging customers to leave reviews especially in the months past Q1 can help increase revenue
-A large of the stock is allocated towards footwear and has a much higher median revenue than non footwear products. The store can pivot to marketing themself's as a sports footwear speciality store or they can diverify their stock to carry more non footwear products and potentially increase revenue that way.

Thank you for reading!